In [2]:
# -*- coding: utf-8 -*-
import os
import json
import re
import string
import pandas as pd
import nltk
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.tokenize import MWETokenizer
from sklearn.decomposition import TruncatedSVD

import numpy as np
np.random.seed(222)

In [6]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/sangwon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sangwon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/sangwon/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
path = '../Data/'
number_of_topic = 80
number_of_words_in_one_topic = 5

In [8]:
datapaths = os.listdir(path)
datapaths.sort()
datapaths

['koreaherald_1517_0.json',
 'koreaherald_1517_1.json',
 'koreaherald_1517_2.json',
 'koreaherald_1517_3.json',
 'koreaherald_1517_4.json',
 'koreaherald_1517_5.json',
 'koreaherald_1517_6.json',
 'koreaherald_1517_7.json']

In [9]:
df = pd.DataFrame()
for p in datapaths:
    with open(path + p, 'r') as f:
        data = json.load(f)

    dataframe = pd.DataFrame.from_dict(data)
    df = df.append(dataframe)

df.columns

Index(['title', ' author', ' time', ' description', ' body', ' section'], dtype='object')

In [10]:
df[' time'][14]

'2017-12-29 16:31:00'

In [11]:
def preprocess_and_split_to_tokens(sentences):
    """
    :param sentences: (array_like) array_like objects of strings.
        e.g., ["I like apples", "I love python3"]
    You can choose the level of pre-processing by yourself.
    The easiest way to start is lowering the case (str.lower).
    :return: array_like objects of array_like objects of tokens.
        e.g., [["I", "like", "apples"], ["I", "love", "python3"]]
    """
    punct_mapping = {"_":" ", "'":" "}
    punct = "/-'?!,#$%\'()*+-/:;<=>@\\^_`{|}~[]" + '""“”’'
    remove = ["[weekender]", "[newsmaker]", "[graphic news]", "(photo news)", "[herald interview]"]
    def clean_special_chars(text, punct, mapping):
        for r in remove:
            text = text.replace(r, "")
        for p in punct:
            text = text.replace(p, f' {p} ')
        for m in mapping:
            text = text.replace(m, mapping[m])
        
        return text

    # tokenized_sentences = []
    modified_sentences = []

    for sentence in sentences:
        sentence = sentence.lower()
        modified_sentence = clean_special_chars(sentence,punct,punct_mapping)
        # tokenized_sentence = modified_sentence.split()
        # tokenized_sentences.append(tokenized_sentence)
        modified_sentences.append(modified_sentence)
        
    # modified_sentences_df = pd.DataFrame(modified_sentences, columns=['title'])
    # return tokenized_sentences
    # return modified_sentences_df
    # modified_sentences_df = pd.Series(modified_sentences)
    return modified_sentences

In [35]:
text = pd.DataFrame()
text = df[['title', ' body', ' time']]
text[' body'] = preprocess_and_split_to_tokens(text[' body'])
text['title'] = preprocess_and_split_to_tokens(text['title'])

# Tokenization
text[' body'] = text.apply(lambda row: word_tokenize(row[' body']), axis=1)
text['title'] = text.apply(lambda row: word_tokenize(row['title']), axis=1)

# Remove stop words
stop = nltk.corpus.stopwords.words('english')
text[' body'] = text[' body'].apply(lambda x: [word for word in x if word not in (stop)])
text['title'] = text['title'].apply(lambda x: [word for word in x if word not in (stop)])

text[' body'] = text[' body'].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])
text['title'] = text['title'].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])

text[' body'] = text[' body'].apply(lambda x: [word for word in x if len(word) > 3])
text['title'] = text['title'].apply(lambda x: [word for word in x if len(word) > 3])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [14]:
from gensim import corpora
dictionary = corpora.Dictionary(text[' body'])
corpus = [dictionary.doc2bow(text) for text in text[' body']]

In [17]:
import gensim
import time
import warnings

from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary

ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = number_of_topic, id2word=dictionary, iterations= 400, passes=40)
topics = ldamodel.print_topics(num_words=number_of_words_in_one_topic)
for topic in topics:
    print(topic)
# coherences=[]
# perplexities=[]
# passes=[]
# warnings.filterwarnings('ignore')
# for i in range(10):
#     if i==0:
#         p=1
#     else:
#         p=i*5
#     tic = time.time()
#     ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = number_of_topic, id2word=dictionary, iterations = 400, passes=p)
#     print('epoch',p,time.time() - tic)
#     # tfidf, corpus 무슨 차이?
#     # lda = models.ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=ntopics, iterations=200000)

#     passes.append(p)
#     cm = CoherenceModel(model=ldamodel, corpus=corpus, coherence='u_mass')
#     coherence = cm.get_coherence()
#     print("Cpherence",coherence)
#     coherences.append(coherence)
#     print('Perplexity: ', ldamodel.log_perplexity(corpus),'\n\n')
#     perplexities.append(ldamodel.log_perplexity(corpus))
# for i in range(10):
#     if i==0:
#         ntopics = 2
#     else:
#         ntopics = 20*i
#     nwords = 100
#     tic = time.time()
#     passes.append(ntopics)
#     lda4 = gensim.models.ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=ntopics, iterations=400, passes=30)
#     print('ntopics',ntopics,time.time() - tic)

#     cm = CoherenceModel(model=lda4, corpus=corpus, coherence='u_mass')
#     coherence = cm.get_coherence()
#     print("Cpherence",coherence)
#     coherences.append(coherence)
#     print('Perplexity: ', lda4.log_perplexity(corpus),'\n\n')
#     perplexities.append(lda4.log_perplexity(corpus))

#ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = number_of_topic, id2word=dictionary, iterations = 400, passes=40)
cm = CoherenceModel(model=ldamodel, corpus=corpus, coherence='u_mass')
coherence = cm.get_coherence()
print("Coherence",coherence)
print('Perplexity: ', ldamodel.log_perplexity(corpus),'\n\n')

(2, '0.111*"children" + 0.098*"child" + 0.089*"care" + 0.054*"health" + 0.051*"pension"')
(54, '0.105*"ambassador" + 0.061*"embassy" + 0.054*"joel" + 0.037*"foreign" + 0.034*"korea"')
(50, '0.023*"information" + 0.017*"attack" + 0.014*"online" + 0.013*"korea" + 0.012*"internet"')
(13, '0.158*"labor" + 0.140*"workers" + 0.060*"wage" + 0.038*"union" + 0.036*"minimum"')
(62, '0.087*"work" + 0.037*"company" + 0.034*"service" + 0.031*"hours" + 0.030*"employees"')
(58, '0.055*"store" + 0.051*"shop" + 0.042*"brand" + 0.040*"purchase" + 0.035*"gangnam"')
(45, '0.027*"case" + 0.026*"disease" + 0.026*"health" + 0.020*"korea" + 0.020*"virus"')
(4, '0.043*"chief" + 0.038*"former" + 0.023*"serve" + 0.022*"sing" + 0.020*"officials"')
(55, '0.095*"court" + 0.027*"seoul" + 0.026*"rule" + 0.025*"samsung" + 0.023*"sentence"')
(56, '0.096*"japan" + 0.051*"japanese" + 0.030*"tokyo" + 0.027*"women" + 0.025*"issue"')
(73, '0.043*"claim" + 0.017*"legal" + 0.015*"decision" + 0.015*"document" + 0.014*"demand"'

In [67]:
#coherence
# from matplotlib import pyplot as plt

# plt.plot(passes, coherences )
# plt.xlabel('Topics')
# plt.ylabel('Coherence')
# plt.show()

In [68]:
#perplexity
# from matplotlib import pyplot as plt

# plt.plot(passes, perplexities)
# plt.xlabel('Topics')
# plt.ylabel('Perplexity')
# plt.show()

In [170]:
# import pyLDAvis.gensim

# pyLDAvis.enable_notebook()
# vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
# pyLDAvis.display(vis)

In [18]:
def make_topictable_per_doc(ldamodel, corpus, texts):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.
        
        if len(doc) == 0:
            topic_table = topic_table.append(pd.Series([-1]), ignore_index=True)
        
        # 모든 문서에 대해서 각각 아래를 수행
        
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num)]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.

            else:
                break
    return(topic_table)

In [19]:
topictable = make_topictable_per_doc(ldamodel, corpus, text[' body'])
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['Doc_num', 'Issue']



,Doc_num,Issue
0,0,3.0
1,1,36.0
2,2,31.0
3,3,39.0
4,4,39.0
...,...,...
23764,23764,59.0
23765,23765,59.0
23766,23766,59.0
23767,23767,39.0


In [1]:
topictable.loc[15, 'Issue']

NameError: name 'topictable' is not defined

In [37]:
topictable.shape

(23769, 2)

# Body LDA 기준으로 겹치는 title 뽑아보기

In [21]:
text_topic_list = []
for j in range(number_of_topic):
    temp_list = [text['title'].iloc[i] for i in range(len(text)) if topictable.loc[i, 'Issue']==j]
    text_topic_list.append(temp_list)

In [22]:
# Find most overllaped sequence
MAX_SEQUENCE_LENGTH = 8
total_dict_list = []
for _ in range(number_of_topic):
    dict_list = []
    for _ in range(MAX_SEQUENCE_LENGTH + 1):
        dict_list.append(dict())
    total_dict_list.append(dict_list)
for k in range(number_of_topic):
    topic_list = text_topic_list[k]
    dict_list = total_dict_list[k]
    for sequence_length in range(1, MAX_SEQUENCE_LENGTH + 1):
    # sequence_length = 1
        for single_topic_list in topic_list:
            # print(sequence_length)
            for i in range( len(single_topic_list) - sequence_length):
        #         print("single_topic_list[i:i+sequence_length]")
        #         print(single_topic_list[i:i+sequence_length])
                if str( single_topic_list[i:i+sequence_length] ) not in dict_list[sequence_length]:
                    dict_list[sequence_length][str( single_topic_list[i:i+sequence_length] )] = 0
                dict_list[sequence_length][str( single_topic_list[i:i+sequence_length] )] += 1

In [23]:
for i in range(number_of_topic):
    print()
    print("Topic"+str(i))
    for j in range(1, MAX_SEQUENCE_LENGTH + 1):
        if len(total_dict_list[i][j]) == 0:
            continue
        key, value = max(total_dict_list[i][j].items(), key=lambda x:x[1])
        print(key)
        print(value)


Topic0
['military']
2
['national', 'news']
1
['former', 'president', 'park']
1
['former', 'president', 'park', 'geun']
1
['former', 'president', 'park', 'geun', 'grill']
1
['korea', 'fine', 'dust', 'particles', 'come', 'china']
1
['labor', 'union', 'leader', 'five', 'year', 'jail', 'term']
1
['labor', 'union', 'leader', 'five', 'year', 'jail', 'term', 'violent']
1

Topic1
['korea']
309
['north', 'korea']
61
['korea', 'test', 'fire']
9
['korea', 'nuclear', 'test', 'site']
4
['chronology', 'north', 'korea', 'missile', 'rocket']
3
['korea', 'likely', 'conduct', 'nuclear', 'missile', 'test']
2
['korea', 'likely', 'conduct', 'nuclear', 'missile', 'test', 'next']
2
['korea', 'likely', 'conduct', 'nuclear', 'missile', 'test', 'next', 'days']
2

Topic2
['care']
9
['child', 'care']
3
['restaurants', 'policy', 'discrimination']
1
['restaurants', 'policy', 'discrimination', 'human']
1
['restaurants', 'policy', 'discrimination', 'human', 'right']
1
['national', 'pension', 'fall', 'short', 'basic'

In [31]:
issue_rank = []
for i in range(number_of_topic):
    print()
    print("Topic"+str(i))
    if len(total_dict_list[i][3]) == 0:
        issue_rank.append([])
        continue
    key, value = max(total_dict_list[i][3].items(), key=lambda x:x[1])
    issue_rank.append(key)


Topic0

Topic1

Topic2

Topic3

Topic4

Topic5

Topic6

Topic7

Topic8

Topic9

Topic10

Topic11

Topic12

Topic13

Topic14

Topic15

Topic16

Topic17

Topic18

Topic19

Topic20

Topic21

Topic22

Topic23

Topic24

Topic25

Topic26

Topic27

Topic28

Topic29

Topic30

Topic31

Topic32

Topic33

Topic34

Topic35

Topic36

Topic37

Topic38

Topic39

Topic40

Topic41

Topic42

Topic43

Topic44

Topic45

Topic46

Topic47

Topic48

Topic49

Topic50

Topic51

Topic52

Topic53

Topic54

Topic55

Topic56

Topic57

Topic58

Topic59

Topic60

Topic61

Topic62

Topic63

Topic64

Topic65

Topic66

Topic67

Topic68

Topic69

Topic70

Topic71

Topic72

Topic73

Topic74

Topic75

Topic76

Topic77

Topic78

Topic79


In [32]:
text_topic_list_with_index = []
for i, topic in enumerate(text_topic_list):
    text_topic_list_with_index.append((i, len(topic)))
text_topic_list_with_index.sort(key=lambda tup: tup[1], reverse = True)
for topic_num, count in text_topic_list_with_index:
    if topic_num == -1:
        continue
    print(" ".join(issue_rank[topic_num]), topic_num, " has ", count, " of news articles")
    print()

[ ' n u c l e a r ' ,   ' t e s t ' ,   ' s i t e ' ] 59  has  2543  of news articles

[ ' l i b e r t y ' ,   ' k o r e a ' ,   ' p a r t y ' ] 31  has  1628  of news articles

[ ' k o r e a ' ,   ' u r g e ' ,   ' k o r e a ' ] 41  has  1225  of news articles

[ ' s e e k ' ,   ' a r r e s t ' ,   ' w a r r a n t ' ] 72  has  961  of news articles

[ ' k o r e a ' ,   ' c h i n a ' ,   ' j a p a n ' ] 53  has  937  of news articles

[ ' k o r e a ' ,   ' c h i l d b i r t h s ' ,   ' c o n t i n u e ' ] 3  has  830  of news articles

[ ' k o r e a ' ,   ' u . s . ' ,   ' c o n d u c t ' ] 74  has  804  of news articles

[ ' d e a t h ' ,   ' d i g n i t y ' ,   ' t r i a l ' ] 67  has  772  of news articles

[ ' p a r k ' ,   ' r e n e w ' ,   ' c a l l ' ] 6  has  746  of news articles

[ ' j a p a n ' ,   ' w a r t i m e ' ,   ' s e x u a l ' ] 56  has  703  of news articles

[ ' v i s i t ' ,   ' k o r e a ' ,   ' n e x t ' ] 75  has  694  of news articles

[ ' u . n . ' ,   ' s e

In [42]:
# sorting dataframe 

#topictable[' body'] =  text[' body'].reset_index()[' body']
text = text.reset_index()
text['Doc_num'] = topictable['Doc_num']
text['Issue'] = topictable['Issue']

#text.sort_values('Issue', inplace = True) 

#topictable.to_csv("../CSV_Data/topictable.csv")
text.to_csv("../CSV_Data/preprocessed_data.csv")
text

,index,title,body,time,Doc_num,Issue
0,0,"[snapshot, multiculturalism, south, korea]","[birthrates, persistently, senior, population,...",2018-01-01 17:07:00,0,3.0
1,1,"[korea, dynamic, 2017]","[north, korea, nuclear, weapons, program, near...",2018-01-01 13:22:00,1,36.0
2,2,"[people, party, members, support, push, merger...","[leader, center, leave, people, party, garner,...",2017-12-31 16:18:00,2,31.0
3,3,"[panamanian, vessel, probe, suspect, supply, k...","[pyeongtaek, south, korea, seize, inspect, pan...",2017-12-31 14:55:00,3,39.0
4,4,"[hong, kong, ship, crew, question, korea, tran...","[crew, hong, kong, register, ship, detain, que...",2017-12-30 15:44:00,4,39.0
...,...,...,...,...,...,...
23764,2765,"[korean, leader, speech, arouse, cautious, opt...","[north, korean, leader, jong, year, speech, he...",2015-01-01 13:36:00,23764,59.0
23765,2766,"[korean, leader, open, inter, korean, summit, ...","[north, korean, leader, jong, thursday, will, ...",2015-01-01 10:05:00,23765,59.0
23766,2767,"[u.s., envoy, call, clearer, communication, ch...","[unite, state, make, thoughts, north, korea, f...",2015-01-01 09:27:00,23766,59.0
23767,2768,"[u.s., impose, sanction, korean, firm]","[unite, state, impose, sanction, north, korean...",2015-01-01 09:25:00,23767,39.0
